In [1]:
import torch 
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from pathlib import Path
import csv
import numpy as np
import cv2
from tqdm import tqdm

from detect_laser import *
from torchinfo import summary
from torchmetrics.classification import BinaryAccuracy

import matplotlib.pyplot as plt
from multiprocessing import Pool, cpu_count
from skimage.feature import peak_local_max

In [2]:
# Prepare data 

data_path = Path("./data")
csv_list = list(data_path.glob("*.csv"))
file_map = dict()
test_map = dict()
i = 0
l = 0
for csv_file in csv_list:   
    with open(csv_file, newline='') as csvfile:

        reader = csv.reader(csvfile, delimiter=',')

        header = next(reader)

        # First 2 of every file will be for testing
        for k in range(2):
            row = next(reader)
            if len(row) > 0:
                test_map[l] = dict()
                test_map[l]["name"] = row[0]
                
                for j in range(1, 3):
                    test_map[l][header[j]] = row[j]

                l+=1
        
        for row in reader:
        
            if len(row) > 0:
                file_map[i] = dict()
                file_map[i]["name"] = row[0]
                
                for j in range(1, 3):
                    file_map[i][header[j]] = row[j]

                i+=1

In [3]:
print(len(file_map))
print(len(test_map))

392
38


In [18]:
# Get the top 1500 peaks in each image

for i in tqdm(file_map):
    
    img = cv2.imread(file_map[i]["name"])
    coordinates = peak_local_max(img[:,:,2], min_distance=20)
    print(f"{file_map[i]['name']}: ", coordinates.shape)
    laser_np = [int(file_map[i]["laser.x"]), int(file_map[i]["laser.y"])]
    laser_big = np.array(coordinates.shape[0]*laser_np)
    print(laser_big.shape)
    break
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_RGB2BGR), cmap="gray")
    plt.plot(coordinates[:, 1], coordinates[:, 0], 'r.')
    plt.show()

    break

  0%|                                                                                                                                      | 0/392 [00:03<?, ?it/s]

data/FSL-01D Box-rectified/P7130212.JPG:  (5950, 2)
(11900,)
